# Exercises due by EOD 2017.11.20 (*a Monday*)

## goal

in this homework assignment we will focus on interacting with relational databases we construct using `aws` `rds`, and `nosql` databases we create using `aws` `dynamodb`.

## method of delivery

as mentioned in our first lecture, the method of delivery may change from assignment to assignment. we will include this section in every assignment to provide an overview of how we expect homework results to be submitted, and to provide background notes or explanations for "new" delivery concepts or methods.

this week you will be submitting the results of your homework via upload to two your own person `s3` homework submission bucket (forget the one we created last week, go back to uploading files to your own bucket).to repeat: use the same homework bucket you had been using for several weeks.

summary:

| exercise | deliverable                                         | method of delivery                 |
|----------|-----------------------------------------------------|------------------------------------|
| 1        | `python` file `dbconnections.py`                    | upload to you `s3` homework bucket |
| 4        | a csv of timing information `bulk_insert_times.csv` | upload to you `s3` homework bucket |
| 6        | `python` file `csvtodynamodb.py`                    | upload to you `s3` homework bucket |

# exercise 1: `boto3` and the `rds` service

remember: the `rds` service is a totally separate concept from the database that is implemented with `rds`. think of it as the chef: it collects the ingredients (one of which is `postgres`) and bakes you a `postgres` cake. we're going to talk to the chef right now instead of the cake.

download the skeleton `python` script here: https://s3.amazonaws.com/shared.rzl.gu511.com/dbconnections.py

fill in the missing pieces to construct a function which can take an `rds` database id (e.g. `rzl-gu511-shared`) and create a `psycopg2` or `sqlalchemy` connection object from just that information.

you can test this file by running the following (the part in brackets is optional) in an environment with `psycopg2` and `sqlalchemy`installed.

```bash
python dbconnections.py --dbid YOUR_RDS_DB_ID [--profile_name YOU_AWS_CONFIGURE_PROFILE]
```

we will test your result by running your file with our `dbid` and `profile name`


##### upload your modified `dbconnections.py` file to the `s3` homework bucket *you* own (from several previous homework assignments)

# exercise 2: creating an `rds` postgres instance

create your own `postgres` `rds` instance using the `aws` `rds` service with the following properties (note: on the "Select Engine" screen, if you click the "Only enable options elegible for RDS free usage tier", most of these options will be filled in for you):

1. engine: `postgresql`
2. engine version: `PostgreSQL 9.6.3-R1`
3. instance class: `db.t2.micro`
4. storage type: SSD
5. storage: 20 GB

be sure to capture the hostname, port, database name, master user name, and master user password.

##### there is nothing to submit for this assignment

# exercise 3: installing `postgres` once for yourself


the goal of this exercise is to replicate what `rds` does for us: create an `ec2` instance and install the `postgres` service. let's start by creating that server.


## 3.1: create the `ec2` server

first, using the `aws` `ec2` service, create a new `ec2` instance that is as close as possible to the underlying `ec2` instance of the `postgres` database instance we created in the previous problem  (that is, it should have the following properties):
    
+ `ami`: amazon linux ami 64 bit (free tier)
+ instance type: `t2.micro`
+ storage: set the type to SSD and the size of the storage to 20 GB

with that done, let's go about installing and configuring `postgres`. 

## 3.2: installing `postgres`

once that server starts up, `ssh` into it. let's start by installing postgres with the following:

```bash
sudo yum install postgresql postgresql-server postgresql-devel postgresql-contrib postgresql-docs
sudo vim /var/lib/pgsql9/data/pg_hba.conf
```

## 3.3: configuring `postgres`

oh fun -- remember `vim`?!

we need to tell the `postgres` service to allow you to log in as the `postgres` user (which is bad practice, but let's do it for fun anyway). toward the bottom of the file you just opened in `vim` there is a pair of lines that look like the following:

```conf
# "local" is for Unix domain socket connections only
local   all             all                                     peer
```

these are saying that for people logged on to your machine (`local`), for all databases (the first `all`) and for all database usernames (the second `all`), use `peer` authentication (that is, get the user's name according to the operating system (so for us, `ec2-user`) and attempt to log them in with that name).

we will loosen up those restrictions by changing the `peer` authentication method to `trust`, which allows anyone who has made it onto our `ec2` instance (`local`) to view any database as any user (`all` and `all`).

```conf
# "local" is for Unix domain socket connections only
local   all             all                                     trust
```

## 3.4: verifying our install

once this edit has been made, you can finally start the server and log in to your own `postgres` database server:

```bash
sudo service postgresql start

# only one user exists right now: postgres
psql -U postgres
```

you're good! feel free to exit that `psql` shell with `\q`

## 3.5: installing `psycopg2`

one last thing: let's install `psycopg2`. execute the following:

```bash
cd /tmp
wget https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
bash Miniconda3-latest-Linux-x86_64.sh

cd
source ~/.bashrc
conda install psycopg2
```

##### there is nothing to submit for this assignment

# exercise 4: comparing bulk `csv` insert methods

there are a few different ways to perform inserts of many records, and they have pretty drastically different performance. the fastest way to insert large quantities of data into a database is *usually* by utilizing a proprietary bulk insert command. for example, in

+ `ms sql`: [a `BULK INSERT` command](https://docs.microsoft.com/en-us/sql/t-sql/statements/bulk-insert-transact-sql)
+ `mysql`: [the `LOAD DATA INFILE` command](https://dev.mysql.com/doc/refman/5.6/en/load-data.html)
+ `oracle`: using a special oracle tool [`sql*loader`](https://docs.oracle.com/cd/B19306_01/server.102/b14215/ldr_concepts.htm) to write a "control" file (to outline the import steps) and import the datafile (this is what you pay hundreds of thousands of dollars for)

`postgres` has implemented both a standard [`sql COPY` command](https://www.postgresql.org/docs/current/static/sql-copy.html) and a `psql` [`\copy` meta command](https://www.postgresql.org/docs/9.2/static/app-psql.html#APP-PSQL-META-COMMANDS-COPY) for this task.

the former (`sql COPY` command) can only be executed by users with appropriate permissions, and *must* be done with a local file. since we are using an unaccessbile `ec2` instance, we are out of luck on that count.

the latter (`\copy psql` command) can be executed by a non-admin user, and can be run remotely. the drawback: it is executed on the client side, so we have to send the entire inserted file over the wire.

let's try out a few different options to see how considerable the speed differences are. first, though, we need to do some setup.


## 4.1: prepping

we will be performing this exercise from the `ec2` server we created above (and on which we installed `postgres`). `ssh` into that `ec2` server and take care of a few setup options.


### 4.1.1: getting data

download the bulk `csv` we intend to upload to our various `postgres` servers:

```bash
cd /tmp
wget https://s3.amazonaws.com/shared.rzl.gu511.com/train_positions/train_positions.csv
chmod a+r train_positions.csv
cd ~
```

### 4.1.2: getting the `python` code

I've written a short snippet of `python` code to

0. mark the starting time
1. create a connection to your `rds` database
2. create a csv dictreader to read lines from `train_positions.csv`
3. iterate through those csv lines and insert each of them one at a time
    1. using a parameterized sql insert statement
4. mark the ending time
5. print the total time it took to insert the records to the screen

download it to your `ec2` instance and review it:

```bash
cd ~
wget https://s3.amazonaws.com/shared.rzl.gu511.com/train_positions/bulkinsert.py
```


### 4.1.3: creating an empty table

you can log into your local `postgres` server with

```bash
psql -U postgres
```

and you can log into your `rds` server with

```bash
psql --host YOUR_RDS_ENDPOINT --port YOUR_RDS_PORT --dbname YOUR_DB_NAME --user YOUR_MASTER_USER_NAME
```

log into each and execute the following `sql` code:

```sql
CREATE TABLE train_positions (   
    carcount real                
    , circuitid real             
    , destinationstationcode text
    , directionnum real          
    , linecode text              
    , secondsatlocation real     
    , servicetype text           
    , trainid text               
    , timestamp timestamp        
);                               
```

you should get

```bash
CREATE TABLE
```

as a result of that command on either server. anything else may be an error.

create this empty table on both servers!



## 4.2: inserting in batches via `python`

the first way we will attempt to insert these records is to use the `psycopg2` library to `INSERT` records in batches of 100 at a time. 

assuming your working directory is the one in which you downloaded the file `bulkinsert.py` above, run the following from your `ec2` server's terminal:

```bash
python bulkinsert.py --host YOUR_RDS_ENDPOINT --port YOUR_RDS_PORT --dbname YOUR_DB_NAME --user YOUR_MASTER_USER_NAME --fcsv /tmp/train_positions.csv
```

note how long this process took.


## 4.3: `psql` `\copy` command

what the previous command did was something pretty common: given a `csv` of records that matches the schema of the table we created, load all of the records. as it happens, this is so common that most relational databases have implemented shortcuts for doing exactly that.

one such shortcut is the `psql` command `\copy`. let's use the `\copy` command to copy this *local* `csv` file to our *remote* `rds` database.

on your `ec2` server, open a `psql` shell connection to your `rds` instance:

```bash
psql --host YOUR_RDS_ENDPOINT --port YOUR_RDS_PORT --dbname YOUR_DB_NAME --user YOUR_MASTER_USER_NAME
```

once in, execute the following `psql` command to turn on query timing (this will measure how long a query takes for us:

```sql
\timing
```

now, execute the `\copy` command:

```sql
\copy train_positions from /tmp/train_positions.csv with delimiter as ',' null as '' csv header;
```

note how long this process took.

after this has been run, clean up after yourself again:

```sql
DELETE FROM train_positions;
```

and then close the `psql` shell session:

```sql
\q
```


## 4.4: `sql` `COPY` command

finally, let's mimic the process of doing a bulk load from a *local* file to a *local* `postgres` server. as I wrote above, this requires a superuser (e.g. user `postgres`) and that the file is on the same computer as the database server. this is exactly the scenario we set up on our `ec2` server.

open a `psql` shell connected to your *local* (to `ec2`) `postgres` server:

```bash
psql -U postgres
```

again, turn on timing:

```sql
\timing
```

and finally, copy the file into the table `train_positions`

```sql
COPY train_positions
FROM '/tmp/train_positions.csv'
WITH (
    FORMAT csv
    , DELIMITER ','
    , NULL ''
    , HEADER TRUE
);
```

note how long this process took.

after this has been run, clean up after yourself again:

```sql
DELETE FROM train_positions;
```

and then close the `psql` shell session:

```sql
\q
```


## 4.5: delivering you results

put the three different processing times you found on this exercise into a `csv` called `bulk_insert_times.csv` which has the following format:

| method                    | time_in_ms |
|---------------------------|------------|
| `psycopg2` batch `INSERT` |            | 
| `\copy`                   |            |
| `COPY`                    |            |

please report all the times you received in `ms` (this is the unit given for `\timing`, as well as the `python` script I wrote).


##### upload `bulk_insert_times.csv` to the `s3` homework bucket *you* own (from several previous homework assignments)

# exercise 5: terminate your `postgres` `ec2` instance

via the `aws` `ec2` web console, terminate the `ec2` instance we created above for your `postgres` database

# exercise 6: using `boto3` to add documents to `dynamodb`

I've been downloading `wmata` metro train position information every 10 seconds since late 2016. at this point, I have about 210,000,000 individual records of train locations. I have collected 5,000,000 of those records and we're going to load them into a `dynamodb` instance and do some quick querying!


## 6.1: getting data

download the bulk `csv` of `wmata` train position data which we intend to upload to our `dynamodb` instance:

```bash
cd /tmp
wget https://s3.amazonaws.com/shared.rzl.gu511.com/train_positions/train_positions.csv
```

## 6.2: converting `csv` records into `json` documents

a single row in a `csv` can easily be converted into a `json` document: simply treat each record as a single `json` objects with keys being column headers and values being the associated record value. for example, a `csv` like

| a | b |
|---|---|
| 0 | 1 |
| 1 | 2 |
| 2 | 4 |
| 3 | 8 |

can be converted into a list of `json` objects

```json
[
    {'a': 0, 'b': 1},
    {'a': 1, 'b': 2},
    {'a': 2, 'b': 4},
    {'a': 3, 'b': 8},
]
```

the `python` `csv.DictReader` class is actually perfrect for this -- that's exactly how it reads `csv` files.

we can create a generator from any `csv` file using the following 5 lines of code:

```python
import csv

def csv_to_json(fcsv):
    with open('fcsv', 'r') as f:
        for record in csv.DictReader(f):
            yield record
```

however, we have to do one more thing: `dynamodb` does not accept null values in `items`, so we should not include the keys in `record` whose values are null:

```python
def csv_to_json(fcsv):
    with open('fcsv', 'r') as f:
        for record in csv.DictReader(f):
            yield {k: v for (k, v) in record.items() if v}

```

this is all overkill -- anywhere we could use this we could also just use `csv.DictReader` in-line.


## 6.3: creating a `dynamodb` table

for the purposes of this exercise, let's just assume that the best choice of hash key is `trainid` (it's not a bad choice: there are many values and it's well distributed), and the best choice of sort key is `timestamp`. for now, leave them both as strings (again, just to keep it simple -- best practice would be to convert them). 

the result would be individual trains ordered by snapshots in time. fun fact -- those trainids are not consistent from one run to the next, so this doesn't actually *mean* anything, it's just for fun.

create a new `dynamodb` table called `TrainPositions` with hash and sort key as described above.


## 6.4: filling in the details

download the `python` skeleton here: https://s3.amazonaws.com/shared.rzl.gu511.com/csvtodynamodb.py

you know the drill -- fill in the `FILL ME IN` sections. the end result should be a function `upload` which can push a `csv` into any `dynamodb` table (provided that the primary key columns for the `dynamodb` table are headers in the `csv` file).


## 6.5: upload the data

you can test your file by running the `upload` function, or invoking the entire `python` script from a `bash` command line:

```bash
python csvtodynamodb.py --fcsv CSV_FILE_PATH --tablename YOUR_DYNAMODB_TABLE_NAME [--profile YOUR_AWS_CONFIG_PROFILE]
```

prove your script works by uploading some records from `train_positions.csv` into `dynamodb`. verify through the web console that records are being added.

*note*: the default provisioned write capacity of 5 units will allow you to write 5 KB per second. our file is 1 GB (1,000,000 KB) so a rough calculation of how long this would take is 200,000 seconds, or 2.3 days. I don't think we're going to write everything that way. feel free to write some number of records and then simply kill the `python` session that is running it `ctrl + c`.


## 6.6: clean up

you can keep the records in that `dynamodb` table if you want, or you can delete the `TrainPositions` table -- it's completely up to you


##### upload your modified version of `csvtodynamodb.py` to your `s3` homework submission bucket